# Load Modules

In [ ]:
import json
import requests
import urllib
import warnings
import pandas as pd
import plotly.express as px
import EDS as eds

warnings.filterwarnings('ignore')

# Define Function to Get FIPS from (lat, lon)

In [ ]:
def find_fips(lat, lon):
    
    # Encode Parameters 
    params = urllib.parse.urlencode(
        {
            'latitude': lat,
            'longitude':lon,
            'format':'json'
        }
    )

    # Get Response From API
    response = requests.get('https://geo.fcc.gov/api/census/block/find?' + params)

    # Parse json in Response
    data = response.json()

    return data['County']['FIPS']

# Calculate Disease Severity

In [ ]:
result = eds.calculate_disease_severity(    
    data_path = "Data.csv",
    info_path = "Info.csv",
    number_of_repeat_year = 4,       
    number_applications_list = [0],
    genetic_mechanistic_list = ["Susceptible"],    
    required_para = eds.required_parameters(),
    output_columns = ["SevMAX"],   
    path_result = None
)

result

# Add FIPS to Results

In [ ]:
result["FIPS"] = result.apply(lambda x: find_fips(lat = x['latitude'], lon = x['longitude']), axis=1)

result

# Drop Duplicate FIPS & Select Maximum

In [ ]:
result = result.sort_values(["FIPS", "SevMAX"]).drop_duplicates(["FIPS"], keep='last').reset_index(drop=True)

# Load Geojson Counties

In [ ]:
with urllib.request.urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
    counties = json.load(response)

# Plot SevMAX

In [ ]:
fig = px.choropleth_mapbox(
    result, 
    geojson=counties, 
    locations='FIPS', 
    color='SevMAX',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=3,
    center={"lat": 37.0902, "lon": -95.7129},
    opacity=0.5,
)

fig.update_layout(
    height=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()